In [1]:

cdfs=pd.read_csv("cdfs.csv", index=False)
ubi_summary=pd.read_csv("ubi_summary.csv", index=False)
nw_quant=pd.read_csv("deciles.csv", index=False)

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import microdf as mdf
import warnings
import plotly.io as pio
import scf
# pio.templates

# Turn off display bar
CONFIG = {"displayModeBar": False}

# Define UBI Center colors
BLUE = "#1976D2"
DARK_BLUE = "#1565C0"
LIGHT_BLUE = "#90CAF9"
GRAY = "#BDBDBD"
BARELY_BLUE = "#E3F2FD"

colors=[BLUE, DARK_BLUE, LIGHT_BLUE, GRAY, BARELY_BLUE]

  Cloning https://github.com/PSLmodels/microdf.git to /tmp/pip-req-build-m8lijvnb
  Running command git clone -q https://github.com/PSLmodels/microdf.git /tmp/pip-req-build-m8lijvnb
  Created wheel for microdf: filename=microdf-0.2.0-cp37-none-any.whl size=26117 sha256=f25935a67c06ea31f6940e45bbf7b134ccafed018a460b167bf3753ec60f3886
  Stored in directory: /tmp/pip-ephem-wheel-cache-p9uj2c8h/wheels/3d/53/af/92e56f83db191b0579d21e8385d61a92a502e66443b23c7e16
Successfully built microdf
Requirement already up-to-date: plotly in /usr/local/lib/python3.7/dist-packages (4.14.3)


The large racial net worth gap traces its roots to slavery, redlining, and other discriminatory policies, and persists largely due to [racial income gaps](https://www.bloomberg.com/news/articles/2019-03-21/how-income-inequality-feeds-the-racial-wealth-gap). In honor of Black History month, we explore how closing part of this income gap with a universal basic income would affect the racial net worth gap, using novel measurements that consider how Black and White families differ across the net worth distribution.

The two most common methods to compare distributions is to compare the mean and median household net worth held by Black families and that held by White families. 


In [3]:

def add_ubi_center_logo(fig, x=0.98, y=-0.14):
  '''returns UBI Center logo on plotly graphs'''
  fig.add_layout_image(
      dict(
          source="https://raw.githubusercontent.com/UBICenter/blog/master/jb/_static/ubi_center_logo_wide_blue.png",
          # See https://github.com/plotly/plotly.py/issues/2975.
          # source="../_static/ubi_center_logo_wide_blue.png",
          xref="paper", yref="paper",
          x=x, y=y,
          sizex=0.12, sizey=0.12,
          xanchor="right", yanchor="bottom"
      )
  )

def common_layout_changes(fig):
  fig.update_layout(
      font=dict(family="Roboto"),
      template=template,
      width=1000,
      height=600
      )
  

def format_helper(fig, 
    title,
    xaxis_title,
    yaxis_title,
    yaxis_ticksuffix="",
    yaxis_tickprefix="", 
    xaxis_tickprefix="",
    hovermode="x"):
  
  fig.update_layout(
      title=title,
      xaxis_title=xaxis_title,
      yaxis_title=yaxis_title,
      yaxis_ticksuffix=yaxis_ticksuffix,
      yaxis_tickprefix=yaxis_tickprefix,
      hovermode=hovermode,
      xaxis_tickprefix=xaxis_tickprefix,
      plot_bgcolor="white",
      legend_title_text="",
      font=dict(family="Roboto"),
      width=1000,
      height=600,
      )

  

  # add ubi center logo
  fig.add_layout_image(
        dict(
            source="https://raw.githubusercontent.com/UBICenter/blog/master/jb/_static/ubi_center_logo_wide_blue.png",
            # See https://github.com/plotly/plotly.py/issues/2975.
            # source="../_static/ubi_center_logo_wide_blue.png",
            xref="paper", yref="paper",
             x=0.98, y=-0.14,
            sizex=0.12, sizey=0.12,
            xanchor="right", yanchor="bottom"
        ))
  
# set template
template='simple_white'

# calculate weighted mean and median grouped by race
params = s.groupby("race2").apply(  
    lambda x: pd.Series(
        {
            "mean": mdf.weighted_mean(x, "networth_pa", "wgt"),
            "median": mdf.weighted_median(x, "networth_pa", "wgt"),
        }
    )
)

# transpose the parameters dataframe
params_T = params.T

def grouped_bar(df):
  # create figure with all weighted median and mean metrics for each race
  fig = go.Figure(
      data=[
          go.Bar(name="Black", x=df.index, y=df["Black"],marker_color=BLUE),
          go.Bar(name="White", x=df.index, y=df["White"],marker_color=GRAY),
      ]
  )
  return fig

fig=grouped_bar(params.T)

format_helper(
    fig,
    title="Racial Wealth Gap - 2019",
    hovermode="x",
    xaxis_title="",
    yaxis_title="Net Worth per Adult",   
    yaxis_tickprefix="$"
)

fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.75))

fig.show(config=CONFIG)

As Matt Bruenig pointed out in his [June 2020 piece](https://www.peoplespolicyproject.org/2020/06/29/the-racial-wealth-gap-is-about-the-upper-classes/), the mean net worth gap is largely shaped by overall net worth inequality, with the vast majority of net worth held by both Black and White households being held by the top decile of each. 

# Plot quintile by race

In [10]:
hovertemplate="%{x:.1f}%"


fig = px.bar(
    nw_quant[nw_quant.variable.isin(['White',"Black"])],
    y="percentile",
    x="value",
    color="variable",
    labels={
        "percentile": "Percent receiving benefit",
        "value": "Share of Within Group Wealth",
    },
    orientation="h",
    barmode="group",
    color_discrete_map={"White": GRAY, "Black": BLUE},
)


fig.update_traces(hovertemplate="Share of Wealth: "+hovertemplate)


# apply formatting
format_helper(
    fig,
    title="Household Inequality by Race",
    hovermode="y",
    xaxis_title="Share of total within-group wealth held by decile",
    yaxis_title="Wealth decile",   
    yaxis_ticksuffix="%"
)


fig.show(config=CONFIG)


While the median tells us where the middle of the distributions lay, it can obscure what is going on throughout the rest of the distribution. 

One tool that we can use to visualize a variable’s distribution in full is with a cumulative distribution function. A cumulative distribution function (CDF) tells us the probability a random variable returns a value less than some specified value. 

To show how to interpret the graph of the CDF, see the following graph showing the distribution of household net worth per adult for all households in the U.S. (Note that while we have logarithmically transformed the x-axis, the y-axis remains linear.) 


In [11]:
# to-dos
# if no s, then we need to melt the df into one I suppose
# 


all_hh = cdfs.loc[cdfs.ubi_mo==0]

# select median. This isn't the weighted median, but it will do for illustrative purposes
# since we're referring to any given pt on x
med = all_hh.loc[all_hh["networth_pa"] == all_hh["networth_pa].median()].max()
med.x = np.log10(med.x)

# plot curve
fig = px.line(x=all_hh.networth_pa_new, y=all_hh.all_share, template=template)
# change line coloar
fig["data"][0]["line"]["color"] = BLUE

# add explanation for arrow pointing to y-axis
fig.add_annotation(
    x=0,
    y=med.y,
    text="Look at y-axis to see share of households <br> whose net worth is below the x-value",
    showarrow=True,
    arrowhead=0,
    ay=-50,
    ax=150,
)

# add dotted line pointing from x-axis to point
fig.add_annotation(
    x=med.x,
    y=0,
    text="For a given net worth on the x-axis",
    showarrow=True,
    arrowhead=0,
    # yshift=-60,
    # xshift=-130,
    ax=160,
    ay=-50,
)

# add red arrow pointing from x-axis to point
fig.add_annotation(
    x=med.x,  # arrows' head
    y=med.y,  # arrows' head
    ax=med.x,  # arrows' tail
    ay=0,  # arrows' tail
    xref="x",
    yref="y",
    axref="x",
    ayref="y",
    text="",  # if you want only the arrow
    showarrow=True,
    arrowhead=2,
    arrowsize=2,
    arrowwidth=1,
    arrowcolor=GRAY,
)

# add arrow pointing from point to y-axis
fig.add_annotation(
    x=0,  # arrows' head
    y=med.y,  # arrows' head
    ax=med.x,  # arrows' tail
    ay=med.y,  # arrows' tail
    xref="x",
    yref="y",
    axref="x",
    ayref="y",
    text="",  # if you want only the arrow
    showarrow=True,
    arrowhead=2,
    arrowsize=2,
    arrowwidth=1,
    arrowcolor=GRAY,
)

fig.update_layout(showlegend=False)

format_helper(
    fig,
    title="Cumulative Distribution of Household Net Worth per Adult in 2019",
    xaxis_title="Net worth (USD)",
    yaxis_title="CDF",
    yaxis_ticksuffix="",
    yaxis_tickprefix="",
    xaxis_tickprefix="$",
    hovermode="x",
)

fig.update_xaxes(
    dict(
        range=[0, 9],
        autorange=False,
        showspikes=False,  # Show spike line for x-axis
        # Format spike
        spikethickness=1,
        spikedash="dot",
        spikecolor="grey",
        spikemode="toaxis",
    ),
    type="log",
)
fig.update_yaxes(
    dict(
        range=[0, 1],
        showspikes=True,
        spikethickness=1,
        spikedash="dot",
        spikecolor="grey",
        spikemode="toaxis",
    )
)


fig.show()


Now, how can we compare the distributions of net worth between Black and White households? The two-sample Kolmogorov-Smirnov (KS) statistical test offers a way to test the difference between two distributions. The KS statistic, or D-statistic, is the maximum absolute vertical distance between two CDFs. One advantage to this statistical method is that it is quite simple to explain visually. The below chart plots the CDF for White and Black households. The dotted line drawn between the two curves shows where the vertical distance between them is at its greatest.

In [14]:
with warnings.catch_warnings():  # ignore "FutureWarning"
    """
    Ignore numpy 'FutureWarning' :
    "Using a non-tuple sequence for multidimensional indexing is deprecated;
    use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be
    interpreted as an array index, `arr[np.array(seq)]`, which will result
    either in an error or a different result."
    """
    warnings.filterwarnings("ignore", category=FutureWarning)


    # Create traces
    fig = go.Figure()

    # create df for baseline
    cdfs_bl = cdfs[cdfs.ubi_mo==0]
    # plot ECDFs
    fig.add_trace(go.Scatter(x=cdfs_bl.networth_pa_new, y=cdfs_bl.white_share, name="White",marker_color=GRAY))
    fig.add_trace(go.Scatter(x=cdfs_bl.networth_pa_new, y=cdfs_bl.black_share, name="Black",marker_color=BLUE))
    
    # define ks_stat
    ks_stat = ubi_summary.loc[ubi_summary.ubi_mo==0, 'ks_stat'].max()


    # find networth where the d-statistic is found
    max_x = ubi_summary.loc[ubi_summary.ubi_mo==0, 'networth_pa_new'].max()
    

    # find networth where abs distance between ecdfs is is @  max
    y1 = ubi_summary.loc[ubi_summary.ubi_mo==0, 'white_share'].max()
    y0 = ubi_summary.loc[ubi_summary.ubi_mo==0, 'black_share'].max()
    


    # # add a dashed line showing where max abs distance between ECDFs is

    dash_names = "Max abs. distance between ECDFs"
    dash_color= 'black'
    fig.add_trace(
        go.Scatter(
            x=[max_x, max_x],
            y=[y0, y1],
            mode="lines+text",
            line={"dash": "dot", "color": dash_color,'width':2},
            visible=True,
            name=dash_names,
            text=["D-statistic: "+str(round(ks_stat,3))+"        ",None],
            textposition="top left",
            showlegend=False
        ))

    fig.add_annotation(
        x=np.log10(max_x), 
        y=(y1+y0)/2,
                text="The Kolmogorov-Smirnov D-statistic <br>"+\
                "is the maximum absolute vertical <br>"+\
                "distance between the two curves",
                showarrow=True,
                arrowhead=3,
                yshift=1,
                ax=200,
                bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor="dark blue",
            opacity=1
                )



    format_helper(fig, 
      title='Cumulative distribution function (CDF) of wealth by race',
      xaxis_title="Net worth (USD)",
      yaxis_title="CDF",
      yaxis_ticksuffix="",
      yaxis_tickprefix="", 
      xaxis_tickprefix="$",
      )

    fig.update_xaxes(type="log")
    fig.show()

KeyError: ignored

This shows us that approximately 65.9% of Black households have a net worth of less than $50,000 per adult, whereas only 33.3% of White households fall below that line. As we will see, the threshold can be at any point along these curves; the KS statistic is the value measuring the size of the gap where the gap is the largest, at 0.326 (the actual net worth value corresponding to the D-statistic was $50,888).

In the following model, we simulate how saving a total of one year’s UBI payments could change the racial net worth gap. We apply a flat tax on all income to fund the UBI program, and subtract each household’s new tax payment from their net worth. 

In the bottom graph, you can see the vertical distance between the two curves at each point along the x-axis from $0 to $2,000 per month.


In [ ]:
# %%timeit


def ecdf_slider(data):
    # Create traces
    fig = go.Figure()

    # create empty list for payment size for sliders
    payments = []
    ks_stats = []
    max_xs = []

    # create 2 subplots
    fig = make_subplots(rows=2, cols=1, shared_xaxes=False)

    for i, step in enumerate(data.ubi_mo.unique()):
        # append payment to list
        payments.append(step)
        # add trace 0
        fig.add_trace(
            go.Scatter(
                x=data.networth_pa_new,
                y=data.white_share,
                name="White",
                visible=False,
                marker_color=GRAY,
            ),
            row=1,
            col=1,
        )
        # add trace 1
        fig.add_trace(
            go.Scatter(
                x=data.networth_pa_new,
                y=data.black_share,
                name="Black",
                visible=False,
                marker_color=BLUE,
            ),
            row=1,
            col=1,
        )
        # append ks-statistics to list
        ks_stat = ubi_summary.loc[ubi_summary.ubi_mo==step,'d_stat_cand'].values[0]
        ks_stats.append(ks_stat)

        # add plot to candidate
        # add trace 2, the d-statistic candidates
        fig.add_trace(
            go.Scatter(
                x=cdfs.networth_pa_new,
                y=cdfs.d_stat_cand,
                mode="lines",
                visible=False,
                name="Absolute Distance Between ECDFs",
                marker_color=LIGHT_BLUE
            ),
            row=2,
            col=1,
        )

        # find networth where the d-statistic
        # find networth where the d-statistic
        max_x = ubi_summary.loc[ubi_summary.ubi_mo==step, 'networth_pa_new'].iloc[0]
        max_xs.append(max_x)

        # find networth where abs distance between ecdfs is is @  max
        y1 = ubi_summary.loc[ubi_summary.ubi_mo==step, 'white_share'].iloc[0]
        y0 = ubi_summary.loc[ubi_summary.ubi_mo==step, 'black_share'].iloc[0]

        dash_names = "Max abs. distance between ECDFs"
        dash_color = "black"
        fig.add_trace(
            go.Scatter(
                x=[max_x, max_x],
                y=[y0, y1],
                mode="lines+text",
                line={"dash": "dot", "color": dash_color, "width": 2},
                visible=False,
                name=dash_names,
                text=["D-statistic: " + str(round(ks_stat, 3)) + "     ", None],
                textposition="top left",
                showlegend=False,
            ),
            row=1,
            col=1,
        )
        fig.add_trace(
            go.Scatter(
                x=[max_x, max_x],
                y=[0, y1],
                mode="lines",
                line={"dash": "dot", "color": dash_color, "width": 2},
                visible=False,
                name=dash_names,
                showlegend=False,
            ),
            row=2,
            col=1,
        )

        fig.add_annotation(
            x=(max_x),
            y=0,
            # text option 1 for general descriptor
            # text="For a given value on the x-axis",
            # text option for net worth descriptor
            text="For a given net worth on the x-axis",
            showarrow=True,
            arrowhead=0,
            # yshift=-60,
            # xshift=-130,
            # ax=160,
            ay=-50,
        )
    # define number of plots in each step
    n = 5
    # Make first n traces visible
    for i in range(0, n):
        fig.data[i].visible = True

    # Create and add slider
    steps = []
    for i in range(0, len(fig.data), n):
        step = dict(
            method="update",
            args=[
                {"visible": [False] * len(fig.data)},
                {
                    "title": "Kolmogorov-Smirnoff D-statistic: "
                    + str(round(ks_stats[i // n], 3))
                },
                # {'annotations[0].text' : 'External %d' % (i // n)},
                {
                    "annotations": dict(
                        x=max_xs[i // n],
                        y=0,
                        text="External %d" % (i // n),
                    )
                },
                # {'shapes':{'visible':[False] * len(fig.layout.shapes)}}
            ],  # layout attribute
            label=("$" + str(payments[i // n])),
        )
        for j in range(0, n):
            step["args"][0]["visible"][i + j] = True  # Toggle i'th trace to "visible"
        steps.append(step)
    sliders = [
        dict(
            active=0,
            currentvalue={"prefix": "Monthly Benefit: "},
            pad={"t": 50},
            steps=steps,
        )
    ]

    fig.update_traces(xaxis="x1")

    # update shapes
    fig.update_shapes(dict(xref="x", yref="y"))

    # fig.update_yaxes(showspikes=True)
    fig.update_layout(
        sliders=sliders,
        xaxis=dict(
            visible=True,
            type="log",
            range=[1, 9],
            autorange=False,
            showspikes=True,  # Show spike line for X-axis
            # Format spike
            spikethickness=2,
            spikedash="dot",
            spikecolor="#999999",
            spikemode="across",
        ),
        title="Empirical Cumlative Density Function (ECDF) of Wealth by Race",
        xaxis_title="Net Worth (USD)",
        yaxis_title="ECDF",
        hovermode="x",
        xaxis_tickprefix="$",
        font=dict(family="Roboto"),
        plot_bgcolor="white",
        height=700,
        width=1000,
        annotations=steps[0]["args"][2]["annotations"],
    )

    fig.update_xaxes(
        visible=True,
        type="log",
        #  autorange=False,
        row=2,
        col=1,
    )
    fig.update_yaxes(
        dict(
            range=[0, 1],
            autorange=False,
            showspikes=False,  # Show spike line for y-axis
            # Format spike
            spikethickness=2,
            spikedash="dot",
            spikecolor="#999999",
            spikemode="toaxis",
        ),
        row=2,
        col=1,
    )

    return fig.show()


with warnings.catch_warnings():  # ignore "FutureWarning"
    """
    Ignore numpy 'FutureWarning' :
    "Using a non-tuple sequence for multidimensional indexing is deprecated;
    use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be
    interpreted as an array index, `arr[np.array(seq)]`, which will result
    either in an error or a different result."
    """
    warnings.filterwarnings("ignore", category=FutureWarning)
    ecdf_slider(cdfs)


It is notable that while the KS D-statistics declines overall, it fluctuates until we get to $500 payments, at which point the D-statistic begins a steady decline to minimum of 0.300 under the $2,000 monthly benefit scenario. This is because the distances between the two curves are much larger around the center of the distributions than around the tails, thus the changes in the KS test are not picking up changes at lower incomes, where the curves were much closer together to begin with.

In our simulations, the net worth point at which the curves are furthest apart increases with the UBI amount. The net worth at which the gap is the biggest rises to just above $121,000, from $50,000 at baseline.


In [ ]:
fig = make_subplots(
    rows=2, 
    cols=1, 
    shared_xaxes=True,
    x_title='UBI per Month',
    subplot_titles=['KS Statistic', 'Most Unequal Net Worth']
    )


fig.add_trace(
    go.Scatter(
        x=ubi_summary["ubi_mo"],
        y=ubi_summary["ks_stat"],
        mode="lines+markers",
        name="ks_stat",
        marker_color=BLUE
    ),row=1,col=1
)
fig.add_trace(
    go.Scatter(
        x=ubi_summary["ubi_mo"],
        y=ubi_summary["most_unequal_networth_pa"],
        mode="lines+markers",
        name="Networth at KS D-stat",
         marker_color=LIGHT_BLUE,
         
    ),row=2, col=1
)

fig.update_layout(
    template=template,
    showlegend=False,
    hovermode="x",
    xaxis_tickprefix="$"
    )

common_layout_changes(fig)
add_ubi_center_logo(fig)

fig.show(config=CONFIG)


Further, without UBI the median White family has $6.45 for every $1 held by the median Black family. Under a $1000 monthly payment, this falls to $3.49 for every $1, and further down to $2.64 with a $2,000 monthly payment. 

The median net worth for White households rises from $115,103 without a UBI to $119,425 under a $1,000 monthly payment, and rises to $127,505 with a $2,000 monthly payment. For Black households, the median net worth rises from $17,853 to $34,183 with a $1,000 benefit, and further to $48,310 with a $2,000 benefit.

It should be noted that the median net worth per adult[<sup>1</sup>](#fn1) rises among White households, it merely rises by less than that of Black households.



<span id="fn1"> Household net worth conventionally adjust for number of adults, see example: [Credit Suisse: Global net worth Report](https://www.credit-suisse.com/about-us/en/reports-research/global-wealth-report.html)</span>

In [ ]:
fig = go.Figure()

hovertemplate= '$%{y:.0f}'

fig.add_trace(
    go.Scatter(
        x=ubi_summary["ubi_mo"],
        y=ubi_summary["black_median_networth_pa"],
        mode="lines+markers",
        name="Black",
        marker_color=BLUE,
        hovertemplate=hovertemplate
        
    )
)
fig.add_trace(
    go.Scatter(
        x=ubi_summary["ubi_mo"],
        y=ubi_summary["white_median_networth_pa"],
        mode="lines+markers",
        name="White",
        marker_color=GRAY,
        hovertemplate=hovertemplate
    )
)


format_helper(
    fig,
    title="Median Net Worth per Adult by Race",
    xaxis_title="UBI per month",
    yaxis_title="Median net worth per adult",
    yaxis_ticksuffix="",
    yaxis_tickprefix="$",
    xaxis_tickprefix="$",
)

fig.show()


Unlike the rising median net worth for White households under the same UBI, mean net worth for White households declines from $557,216 without any UBI to $555,135 under a $1,000 monthly payment, and falls further to $553,054 with a $2000 monthly payment. For Black households, the mean net worth rises from $97,167 to $104,768 with a $1,000 benefit, and further to $112,369 with a $2,000 benefit.

In [ ]:
fig = go.Figure()

hovertemplate= '$%{y:.0f}'

fig.add_trace(
    go.Scatter(
        x=ubi_summary["ubi_mo"],
        y=ubi_summary["black_mean_networth_pa"],
        mode="lines+markers",
        name="Black",
        marker_color=BLUE,
        hovertemplate=hovertemplate
        
    )
)
fig.add_trace(
    go.Scatter(
        x=ubi_summary["ubi_mo"],
        y=ubi_summary["white_mean_networth_pa"],
        mode="lines+markers",
        name="White",
        marker_color=GRAY,
        hovertemplate=hovertemplate
    )
)


format_helper(
    fig,
    title="Mean Net Worth per Adult by Race",
    xaxis_title="UBI per month",
    yaxis_title="Mean net worth per adult",
    yaxis_ticksuffix="",
    yaxis_tickprefix="$",
    xaxis_tickprefix="$",
)

fig.show()

Finally, we look at the share of households that fall below $50,000 per adult, the point at which those two curves were further apart in our baseline scenario. 

In [ ]:
fig = go.Figure()
hovertemplate= '$%{y:.0f}'

fig.add_trace(
    go.Scatter(
        x=ubi_summary["ubi_mo"],
        y=ubi_summary['Black_share_above_50k']*100,
        mode="lines+markers",
        name="Black",
        marker_color=BLUE,
        hovertemplate=pct_str_format
    )
)
fig.add_trace(
    go.Scatter(
        x=ubi_summary["ubi_mo"],
        y=ubi_summary['White_share_above_50k']*100,
        mode="lines+markers",
        name="White",
        marker_color=GRAY,
        hovertemplate=pct_str_format
    )
)

format_helper(
    fig,
    title="Share of Households with Net Worth above $50k by Race",
    xaxis_title="UBI per month",
    yaxis_title="Share of households with net worth above $50k",
    yaxis_ticksuffix="%",
    yaxis_tickprefix="",
    xaxis_tickprefix="$",
)

fig.show()

The share of Black households with a per adult networth of over $50,000 rises from 65.9% to 61.2% when we raise the monthly UBI to $1,000 monthly, and falls further to 51.7% when we raise the benefit to $2,000 monthly.

In the scenarios we explore, the outcomes for Black households improve in the share of households with net worth per adult below $50,000 declines, median household net worth increases. In the final plot below, you can explore the relative changes in all of the metrics we’ve chosen.


# Create Plot With Sliders - `ecdf_slider`

In [ ]:
# create summary plot
fig=go.Figure()

def trace(col, chg_col, name, string, color):
  """
    Args:
        col: Column name.
        chg_col: Column name of change.
        name: Name of column for printing.
        string: Formatting of customdata[1].
        color: Line color.
    """
  fig.add_trace(
      go.Scatter(
          x=ubi_summary["ubi_mo"],
          y=ubi_summary[chg_col],
          mode="lines+markers",
          name=name,
          marker_color=color,
          marker=dict(size=5),
          customdata=np.stack((ubi_summary.ubi_mo, ubi_summary[col]), axis=-1),
            hovertemplate=
            # "<br>UBI/mo: %{x}<br>"
            # + "UBI: $%{customdata[0]: .0f}<br>"
            # + 
            name
            + ": "
            + string
            + "<br>"
            + "Percent change: %{y:.0f}%<br>",
          
      )
  )




trace(
    "white_mean_nw_as_pct_of_mean_black",
    'white_mean_nw_as_pct_of_mean_black_deltap',
    'White/Black mean net worth',
    "%{customdata[1]: .1f}%",
    DARK_BLUE)
# median net worth
trace(
    "white_median_nw_as_pct_of_median_black",
    'white_median_nw_as_pct_of_median_black_deltap',
    'White/Black median net worth',
    "%{customdata[1]: .1f}%",
    GRAY
    )
trace(
    "ks_stat",
    'ks_stat_deltap',
    'KS Statistic',
    "%{customdata[1]: .3f}",
    LIGHT_BLUE
    )
trace(
    "White_share_above_50k_pct_of_black",
    'White_share_above_50k_pct_of_black_deltap',
    'Share of Households over $50k: White/Black',
    "%{customdata[1]: .1f}%",
    BLUE
    )


format_helper(
    fig,
    title="How Different Measures of Racial Wealth Gap Change with UBI",
    xaxis_title="UBI per month",
    yaxis_title="Percent change in measures over baseline",
    yaxis_ticksuffix="%",
    yaxis_tickprefix="",
    xaxis_tickprefix="$",
)

# fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.75))

fig.show(config=CONFIG)

# fig.show()

However, we should note that this simulation does not show how these effects could grow over many years, nor does it account for differences in the composition of assets and debts each held by each household. [<sup>2</sup>](#fn2)

The power of a Universal Basic Income, however, goes beyond what we have shown in our model. The UBI approach does not rely on the discretion of employers - private or public - to conduct just hiring practices, the willingness of regulators to faithfully enforce anti-discrimination laws, nor the ability of individuals to navigate opaque and counterproductive eligibility criteria of safety net programs that have been long maligned and often underfunded. Above all else, the advantage of a UBI over other approaches is that once in place it will not rely on decades of good faith effort by those in power to keep it in place, but the support of all of those that will benefit. 





<span id="fn2"> Future researchers should explore alternative statistical tools that build off the KS test, such as the Cramer Von-Mises Test, which is more sensitive to outliers. 
</span>